In [1]:
# Install required libraries
!pip3 install requests
!pip3 install beautifulsoup4
!pip3 install pandas
!pip3 install fuzzywuzzy
!pip3 install python-Levenshtein

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import quote
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import get_close_matches


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('csvs/parsed_output.csv')
# Display the first few rows of the DataFrame
df.head()

,marca,tipo producto,Model Name,Engine,Drive,Transmission,Year
0,changan,camioneta,hunter 1.9l comfort,NaN,NaN,NaN,NaN
1,changan,camioneta,hunter 1.9l comfort,NaN,4X4,NaN,NaN
2,changan,camioneta,hunter 1.9l elite,NaN,4X4,NaN,NaN
3,changan,camioneta,hunter 1.9l luxury,NaN,NaN,NaN,NaN
4,changan,camioneta,hunter 1.9l luxury,NaN,4X4,NaN,NaN


In [3]:
import re
import time
import requests

# Define a helper function to handle retries
def make_request_with_retry(url, retries=10, delay=2):
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response
        except requests.exceptions.HTTPError as err:
            if response.status_code == 500:
                if attempt < retries - 1:
                    print(f"Server error (500) encountered at {url}. Waiting {delay} second(s) before retrying...")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Failed to access {url} after {retries} attempts: {err}")
                    return None
            else:
                print(f"Failed to access {url}: {err}")
                return None
        except Exception as e:
            print(f"An error occurred while accessing {url}: {e}")
            return None

def normalize_string(s):
    s = s.lower()
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def find_base_model_and_version(model_name):
    descriptors = ['advance', '3', 'row', 'e-power', 'exclusive', 'sense', 'le', 'xle', 'limited',
                   'luxury', 'pro', 'elite', 'glx', 'at', 'mt', '4x2', '4x4', 'awd', '2wd', 'dct',
                   'cvt', 'td', 'tdi', 'tdci', 's', 'se', 'lt', 'ltz', 'ls', 'premium', 'plus',
                   'xt', 'xtronic', 'sunroof', 'diesel', 'gasolina', 'elegance', 'hibrido', 'hev',
                   'hybrid', 'comfort', 'deluxe', 'gt', 'sport', 'xl', 'xlt', 'xse', 'p4x']
    words = normalize_string(model_name).split()
    base_model_words = [word for word in words if word not in descriptors]
    base_model_name = ' '.join(base_model_words)
    version_words = [word for word in words if word in descriptors]
    version_name = ' '.join(version_words)
    return base_model_name.strip(), version_name.strip()

def get_best_match(query, choices, cutoff=0.6):
    matches = get_close_matches(query, choices, n=1, cutoff=cutoff)
    if matches:
        return matches[0]
    else:
        # Attempt substring matching
        for choice in choices:
            if query in choice or choice in query:
                return choice
    return None

#### Guardamos el nombre de las marcas

In [5]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('csvs/parsed_output.csv')

# Extract the unique 'marca's
unique_marcas = df['marca'].drop_duplicates()

# Create a new DataFrame with the unique 'MARCA's
unique_marcas_df = pd.DataFrame(unique_marcas, columns=['macra'])

# Save the unique 'MARCA's to a new CSV file
unique_marcas_df.to_csv('unique_marcas.csv', index=False)


In [9]:
# Leer el archivo unique_marcas.csv
marcas_df = pd.read_csv('csvs/unique_marca_modelo_by_tipo_producto.csv')
marcas_df = marcas_df[['marca']].drop_duplicates()

# Lista para almacenar los datos
models_data = []

# Iterar sobre cada marca
for index, row in marcas_df.iterrows():
    manufacturer = str(row['marca']).lower()
    print(f"Procesando marca: {manufacturer}")
    
    # Construir la URL
    base_url = 'https://www.autocosmos.cl/catalogo/vigente/'
    manufacturer_normalized = normalize_string(manufacturer).replace(' ', '-')
    manufacturer_url = base_url + manufacturer_normalized
    
    # Realizar la solicitud HTTP con reintentos
    response = make_request_with_retry(manufacturer_url)
    if response is None:
        continue  # Saltar a la siguiente iteración si la solicitud falló
    
    # Analizar la página de la marca
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar la sección de modelos
    models_section = soup.find('section', class_='section m-brand')
    if models_section:
        model_links = models_section.find_all('a', href=True)
    else:
        print(f"No se encontraron modelos para {manufacturer}")
        continue
    
    # Extraer nombres de modelos y URLs
    for link in model_links:
        model_name = link.get('title', '').strip()
        model_href = link['href']
        # Construir la URL completa del modelo
        full_model_url = 'https://www.autocosmos.cl' + model_href
        
        # Almacenar los datos
        models_data.append({
            'MARCA': manufacturer.title(),
            'Model Name': model_name,
            'Model URL': full_model_url
        })

# Convertir la lista a un DataFrame
models_df = pd.DataFrame(models_data)

# Guardar en un archivo CSV
models_df.to_csv('csvs/models_by_marcas.csv', index=False)

# Mostrar una muestra de los datos
print(models_df.head())

Procesando marca: changan
Procesando marca: chevrolet
Procesando marca: dfsk
Procesando marca: dongfeng
Procesando marca: fiat
Procesando marca: ford
Procesando marca: foton
Procesando marca: great wall
Procesando marca: honda
Procesando marca: jac
Procesando marca: jmc
Procesando marca: karry
Procesando marca: kia
Procesando marca: mahindra
Procesando marca: maxus
Procesando marca: mazda
Procesando marca: mitsubishi
Procesando marca: nissan
Procesando marca: peugeot
Procesando marca: ram
Procesando marca: renault
Procesando marca: ssangyong
Procesando marca: suzuki
Procesando marca: toyota
Procesando marca: volkswagen
Procesando marca: zna
Procesando marca: baic
Procesando marca: byd
Procesando marca: chery
Procesando marca: citroen
Procesando marca: hyundai
Procesando marca: mg
Procesando marca: exeed
Procesando marca: gac
Procesando marca: haval
Procesando marca: jeep
Procesando marca: jetour
Procesando marca: kaiyi
Procesando marca: opel
Procesando marca: subaru
Procesando marca: f

In [10]:
df = pd.read_csv('csvs/models_by_marcas.csv')

# Function to clean the Model Name
def clean_model_name(row):
    marca = normalize_string(row['MARCA'].strip())
    model_name = row['Model Name'].strip()
    model_name_normalized = normalize_string(model_name)
    
    # Remove the manufacturer name from the model name
    pattern_marca = re.compile(r'\b' + re.escape(marca) + r'\b', re.IGNORECASE)
    model_name_cleaned = pattern_marca.sub('', model_name_normalized).strip()
    
    # Remove the phrase 'nuevos, precios del catálogo y cotizaciones.'
    phrase_to_remove = 'nuevos, precios del catalogo y cotizaciones.'
    model_name_cleaned = model_name_cleaned.replace(phrase_to_remove, '').strip()
    
    pattern_marca_original = re.compile(r'\b' + re.escape(row['MARCA']) + r'\b', re.IGNORECASE)
    model_name_original_cleaned = pattern_marca_original.sub('', row['Model Name']).strip()
    # Then remove the phrase (case-insensitive)
    pattern_phrase = re.compile(re.escape('nuevos, precios del catálogo y cotizaciones.'), re.IGNORECASE)
    model_name_original_cleaned = pattern_phrase.sub('', model_name_original_cleaned).strip()
    
    return model_name_original_cleaned

# Apply the cleaning function to the 'Model Name' column
df['Model Name'] = df.apply(clean_model_name, axis=1)

# Save the cleaned DataFrame to a new CSV
df.to_csv('models_by_marcas_cleaned.csv', index=False)

In [5]:
import pandas as pd
import unicodedata
import re
from fuzzywuzzy import fuzz

# Load CSV files
df1 = pd.read_csv('csvs/models_by_marcas_cleaned.csv')
df2 = pd.read_csv('csvs/parsed_output.csv')

# Standardize column names to lowercase
df1.columns = df1.columns.str.lower().str.strip()
df2.columns = df2.columns.str.lower().str.strip()

# Ensure correct data types and fill missing values
columns_to_normalize = ['marca', 'model name']
for col in columns_to_normalize:
    df1[col] = df1[col].astype(str).fillna('')
    df2[col] = df2[col].astype(str).fillna('')

# Function to normalize strings
def normalize_string(s):
    s = str(s).lower()
    s = ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
    s = re.sub(r'[^\w\s]', '', s)  # Remove punctuation
    s = re.sub(r'\s+', ' ', s).strip()  # Remove extra spaces
    return s

# Normalize 'model name' and 'marca'
for df in [df1, df2]:
    df['model name simplified'] = df['model name'].apply(normalize_string)
    df['marca simplified'] = df['marca'].apply(normalize_string)

# Function to match model names
def match_model(row):
    marca = row['marca simplified']
    model_name = row['model name simplified']
    df1_marca = df1[df1['marca simplified'] == marca]
    if df1_marca.empty:
        return None

    # Try exact match first
    exact_matches = df1_marca[df1_marca['model name simplified'] == model_name]
    if not exact_matches.empty:
        return exact_matches.iloc[0]['model url']

    # Try substring match
    substring_matches = df1_marca[df1_marca['model name simplified'].apply(lambda x: x in model_name or model_name in x)]
    if not substring_matches.empty:
        return substring_matches.iloc[0]['model url']

    # Try fuzzy matching
    df1_marca['score'] = df1_marca['model name simplified'].apply(lambda x: fuzz.token_set_ratio(model_name, x))
    best_match = df1_marca.loc[df1_marca['score'].idxmax()]
    if best_match['score'] >= 80:  # Adjusted threshold
        return best_match['model url']
    else:
        return None
    
def match_model_2(row):
    marca = row['marca simplified']
    model_name = row['model name simplified']
    
    # Filter df1 by the same brand
    df1_marca = df1[df1['marca simplified'] == marca]
    if df1_marca.empty:
        return None

    # Compute fuzzy match scores for all models in the filtered df1
    df1_marca['score'] = df1_marca['model name simplified'].apply(lambda x: fuzz.token_set_ratio(model_name, x))
    
    # Select the top-scoring match
    best_match = df1_marca.loc[df1_marca['score'].idxmax()]
    return best_match['model url']


# Apply the matching function to df2
# df2['model url'] = df2.apply(match_model, axis=1)

# Save the updated df2
# df2.to_csv('csvs/parsed_output_with_urls.csv', index=False)

# Apply the matching function to df2
df2['model url'] = df2.apply(match_model_2, axis=1)

# Save the updated df2
df2.to_csv('csvs/parsed_output_with_urls_2.csv', index=False)

# Print the unmatched models
unmatched = df2[df2['model url'].isnull()]
print(unmatched)

        marca tipo producto model name  engine drive transmission    year  \
1532  farizon        furgon        v6e     NaN   NaN           AT  2024.0   

     model name simplified marca simplified model url  
1532                   v6e          farizon      None  


In [8]:
import pandas as pd

# Read the CSV file with Model URLs
df = pd.read_csv('csvs/parsed_output_with_urls_2.csv')

# Ensure 'Model URL' column is of string type and fill NaNs
df['model url'] = df['model url'].astype(str).fillna('')

# Drop empty URLs and get unique URLs
unique_urls = df['model url'][df['model url'] != ''].unique()

# Create a DataFrame from the unique URLs
df_unique_urls = pd.DataFrame(unique_urls, columns=['model url'])

# Save the unique URLs to a new CSV file
df_unique_urls.to_csv('csvs/unique_model_urls.csv', index=False)

print("Unique Model URLs have been saved to 'unique_model_urls.csv'.")

Unique Model URLs have been saved to 'unique_model_urls.csv'.


In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# Read the unique URLs
df_urls = pd.read_csv('csvs/unique_model_urls.csv')

# Initialize a list to hold scraped data
scraped_data = []

max_retries = 10  # Maximum number of retries for 500 errors

for index, row in df_urls.iterrows():
    url = row['model url']
    print(f"Scraping URL: {url}")
    
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url, timeout=None)
            if response.status_code == 500:
                print(f"Received 500 error on {url}, retrying... ({attempt + 1}/{max_retries})")
                attempt += 1
                time.sleep(1)  # Sleep before retrying
                continue
            else:
                response.raise_for_status()
                break  # Successful response, exit retry loop
        except requests.exceptions.RequestException as e:
            print(f"An error occurred for {url}: {e}")
            break  # Break on any request exception other than 500 error
    else:
        # This else executes if the loop completes without a break (after max retries)
        print(f"Failed to retrieve {url} after {max_retries} attempts.")
        continue

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table containing the versions
    version_table = soup.find('table', class_='table')
    if not version_table:
        print(f"No version table found at {url}")
        continue
    
    # Find all rows in the table body
    tbody = version_table.find('tbody')
    if not tbody:
        print(f"No table body found at {url}")
        continue

    rows = tbody.find_all('tr', itemprop='offers')
    versions = []
    for row in rows:
        # Find the meta tag with itemprop="name"
        meta_name = row.find('meta', itemprop='name')
        a_tag = row.find('a', itemprop='url')
        href = a_tag.get('href', '') if a_tag else ''
        if meta_name:
            version_name = meta_name.get('content', '')
            version_name = re.sub(r'\s*\(\d{4}\)\s*', '', version_name)
            full_url = "https://www.autocosmos.cl" + href
            versions.append([version_name, full_url])
    
    scraped_data.append({
        'model url': url,
        'versions': versions
    })
    
    time.sleep(3)

# Save the scraped data to a CSV
df_scraped = pd.DataFrame(scraped_data)

# Convert 'Versions' from list of lists to a string
def versions_to_string(versions_list):
    return '; '.join([f"{v[0]}: {v[1]}" for v in versions_list])

df_scraped['versions'] = df_scraped['versions'].apply(versions_to_string)

df_scraped.to_csv('csvs/model_versions.csv', index=False)

print("Scraping completed. Data saved to 'model_versions.csv'.")

Scraping URL: https://www.autocosmos.cl/catalogo/vigente/changan/hunter
An error occurred for https://www.autocosmos.cl/catalogo/vigente/changan/hunter: 403 Client Error: Ip Forbidden for url: https://www.autocosmos.cl/catalogo/vigente/changan/hunter
No version table found at https://www.autocosmos.cl/catalogo/vigente/changan/hunter
Scraping URL: https://www.autocosmos.cl/catalogo/vigente/changan/md201-pick-up
An error occurred for https://www.autocosmos.cl/catalogo/vigente/changan/md201-pick-up: 403 Client Error: Ip Forbidden for url: https://www.autocosmos.cl/catalogo/vigente/changan/md201-pick-up
No version table found at https://www.autocosmos.cl/catalogo/vigente/changan/md201-pick-up
Scraping URL: https://www.autocosmos.cl/catalogo/vigente/changan/md201-cargo-box
An error occurred for https://www.autocosmos.cl/catalogo/vigente/changan/md201-cargo-box: 403 Client Error: Ip Forbidden for url: https://www.autocosmos.cl/catalogo/vigente/changan/md201-cargo-box
No version table found a

KeyboardInterrupt: 

In [17]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Read the CSV files
df_parsed = pd.read_csv('csvs/parsed_output_with_urls_2.csv')
df_versions = pd.read_csv('csvs/model_versions.csv')

# Function to parse the 'Versions' column
def parse_versions_string(versions_str):
    version_list = []
    if pd.isna(versions_str) or versions_str.strip() == '':
        return version_list
    versions = versions_str.split(';')
    for v in versions:
        v = v.strip()
        if not v:
            continue
        if ':' in v:
            name, url = v.split(':', 1)
            version_list.append([name.strip(), url.strip()])
        else:
            version_list.append([v.strip(), ''])
    return version_list

# Apply the parsing function to 'Versions' column
df_versions['versions'] = df_versions['versions'].apply(parse_versions_string)

# Function to simplify strings
def simplify_string(s):
    s = str(s).lower()
    s = re.sub(r'[^a-z0-9\s]', '', s)  # Remove special characters
    s = re.sub(r'\s+', ' ', s)  # Replace multiple spaces with single space
    s = s.strip()
    return s

# Simplify relevant columns in df_parsed
df_parsed['Model Name Simplified'] = df_parsed['Model Name Simplified'].apply(simplify_string)
df_parsed['Engine Simplified'] = df_parsed['Engine'].apply(lambda x: simplify_string(x))
df_parsed['Drive Simplified'] = df_parsed['Drive'].apply(lambda x: simplify_string(x))
df_parsed['Transmission Simplified'] = df_parsed['Transmission'].apply(lambda x: simplify_string(x))

# Function to extract attributes from version name
def extract_attributes(version_name):
    version_name = version_name.lower()
    # Engine extraction
    engine_match = re.search(r'(\d+\.\d+|\d+)(l|t)?', version_name)
    engine = engine_match.group(1) if engine_match else ''
    # Drive extraction
    drive_match = re.search(r'(4x2|4x4|awd|fwd|rwd)', version_name)
    drive = drive_match.group(1) if drive_match else ''
    # Transmission extraction
    transmission_match = re.search(r'\b(mt|at|manual|aut|automatic)\b', version_name)
    transmission = transmission_match.group(1) if transmission_match else ''
    simplified_name = simplify_string(version_name)
    return simplified_name, engine, drive, transmission

# Initialize a list to hold the matched data
matched_data = []

for index, row in df_parsed.iterrows():
    model_url = row['Model URL']
    model_name_simplified = row['Model Name Simplified']
    engine_simplified = row['Engine Simplified']
    drive_simplified = row['Drive Simplified']
    transmission_simplified = row['Transmission Simplified']

    # Get the versions corresponding to this model URL
    versions_row = df_versions[df_versions['Model URL'] == model_url]
    if versions_row.empty:
        matched_version = None
    else:
        versions_list = versions_row.iloc[0]['Versions']
        best_score = -1  # Initialize to -1 to handle zero score cases
        best_match = None

        # If there is only one version, match it directly
        if len(versions_list) == 1:
            version = versions_list[0]
            version_name, version_url = version
            matched_version = {
                'Version URL': version_url,
            }
        else:
            for version in versions_list:
                version_name, version_url = version
                # Extract attributes
                version_name_simplified, v_engine, v_drive, v_transmission = extract_attributes(version_name)

                # Calculate matching score
                name_score = fuzz.token_sort_ratio(model_name_simplified, version_name_simplified)
                engine_score = fuzz.partial_ratio(engine_simplified, v_engine) if engine_simplified and v_engine else 0
                drive_score = 100 if drive_simplified == v_drive and drive_simplified else 0
                transmission_score = 100 if transmission_simplified == v_transmission and transmission_simplified else 0

                total_score = name_score + engine_score + drive_score + transmission_score

                if total_score > best_score:
                    best_score = total_score
                    best_match = {
                        'Version URL': version_url,
                    }

            matched_version = best_match

    # Ensure that all keys are present even if no match is found
    if not matched_version:
        matched_version = {
            'Version URL': None,
        }

    # Append matched version to the row
    matched_data.append({
        **row.to_dict(),
        **matched_version
    })

# Create a DataFrame from the matched data
df_matched = pd.DataFrame(matched_data)

# Remove duplicates based on 'Model Name', 'Engine', 'Drive', 'Transmission'
df_matched = df_matched.drop_duplicates(subset=['Model Name', 'Engine', 'Drive', 'Transmission'])

# Save the matched data to a new CSV
df_matched.to_csv('csvs/matched_versions.csv', index=False)

print("Matching completed. Data saved to 'matched_versions.csv'.")


Matching completed. Data saved to 'matched_versions.csv'.


### Correr esta celda para obtener el csv final "scraped_data.csv"

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

# Read the matched_versions.csv file
df_matched = pd.read_csv('csvs/matched_versions.csv')

# Initialize a list to hold the output data
output_data = []

for index, row in df_matched.iterrows():
    # Extract necessary fields and handle 'nan' values
    marca = str(row['marca']) if not pd.isna(row['marca']) else ''
    model_name = str(row['model name']) if not pd.isna(row['model name']) else ''
    engine = str(row['Engine']) if not pd.isna(row['Engine']) else ''
    drive = str(row['Drive']) if not pd.isna(row['Drive']) else ''
    transmission = str(row['Transmission']) if not pd.isna(row['Transmission']) else ''
    year = str(row['Year']) if not pd.isna(row['Year']) else ''
    version_url = str(row['Version URL']) if not pd.isna(row['Version URL']) else ''

    # Create the MODEL column and remove any 'nan' substrings
    model = ' '.join([marca, model_name, engine, drive, transmission, year])
    model = model.replace('nan', '').strip()

    # Remove any extra spaces
    model = ' '.join(model.split())

    # Print progress
    print(f"Processing model: {model}")

    # Check if version_url is valid
    if not version_url or pd.isna(version_url) or version_url.lower() == 'nan':
        print(f"Skipping {model} due to invalid Version URL")
        continue  # Skip to next row

    # Initialize retry parameters
    retries = 0
    max_retries = 20
    success = False

    while retries < max_retries and not success:
        try:
            response = requests.get(version_url)
            if response.status_code == 200:
                success = True
                html_content = response.content
            elif response.status_code == 500:
                retries += 1
                time.sleep(1)  # Wait for 1 second before retrying
                continue
            else:
                # Handle other HTTP errors
                print(f"Error fetching {version_url}: HTTP {response.status_code}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Request exception for {version_url}: {e}")
            break

    if not success:
        print(f"Failed to fetch {version_url} after {retries} retries.")
        continue  # Skip to next row

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Initialize a dictionary to hold the attributes
    attributes = {}

    # Scrape 'Especificaciones técnicas' section
    especificaciones_section = soup.find('section', {'id': 'especificaciones'})
    if especificaciones_section:
        # Extract all tables within this section
        tables = especificaciones_section.find_all('table', {'class': 'ficha'})
        for table in tables:
            # Get the caption (category)
            caption_tag = table.find('caption')
            category = caption_tag.get_text(strip=True) if caption_tag else 'Unknown'

            # For each row in the table
            rows = table.find_all('tr')
            for row_tr in rows:
                cols = row_tr.find_all('td')
                if len(cols) == 2:
                    attr_name = cols[0].get_text(strip=True)
                    attr_value = cols[1].get_text(strip=True)
                    # Prefix the attribute name with the category to avoid duplicates
                    key = f"{category} - {attr_name}"
                    attributes[key] = attr_value
    else:
        print(f"No 'Especificaciones técnicas' section found in {version_url}")
        # We can continue to scrape 'Equipamiento' even if 'Especificaciones técnicas' is missing

    # Scrape 'Equipamiento' section
    equipamiento_section = soup.find('section', {'id': 'equipamiento'})
    if equipamiento_section:
        # Extract all tables within this section
        tables = equipamiento_section.find_all('table', {'class': 'ficha'})
        for table in tables:
            # Get the caption (category)
            caption_tag = table.find('caption')
            category = caption_tag.get_text(strip=True) if caption_tag else 'Unknown'

            # For each row in the table
            rows = table.find_all('tr')
            for row_tr in rows:
                cols = row_tr.find_all('td')
                if len(cols) == 2:
                    attr_name = cols[0].get_text(strip=True)
                    attr_value = cols[1].get_text(strip=True)
                    # Prefix the attribute name with the category to avoid duplicates
                    key = f"{category} - {attr_name}"
                    attributes[key] = attr_value
    else:
        print(f"No 'Equipamiento' section found in {version_url}")

    # Create a dictionary for this row
    data_row = {
        'MODEL': model,
        'MARCA': marca,
        'Model Name': model_name,
        'Engine': engine,
        'Drive': drive,
        'Transmission': transmission,
        'Year': year,
        'Version URL': version_url
    }
    data_row.update(attributes)

    # Replace 'nan' with empty string in data_row values
    data_row = {k: ('' if v == 'nan' else v) for k, v in data_row.items()}

    # Append to output_data
    output_data.append(data_row)

# Create a DataFrame from the output data
df_output = pd.DataFrame(output_data)

# Drop columns that are entirely empty
df_output.dropna(axis=1, how='all', inplace=True)

# Replace NaN values with empty strings
df_output.fillna('', inplace=True)

# Save to a new CSV file
df_output.to_csv('csvs/scraped_data.csv', index=False)

print("Data scraping completed. Results saved to 'scraped_data.csv'.")

Processing model: CHANGAN HUNTER COMFORT 2.0T 4X2 MT 2024
Processing model: CHANGAN HUNTER COMFORT 2.0T 4X4 MT 2024
Processing model: CHANGAN HUNTER ELITE SPORT 2.0T 4X4 MT 2024
Processing model: CHANGAN HUNTER LUXURY 2.0T 4X2 MT 2024
Processing model: CHANGAN MD 301 AC ESP MT 2024
Skipping CHANGAN MD 301 AC ESP MT 2024 due to invalid Version URL
Processing model: CHANGAN MD201 MT 2024
Processing model: CHANGAN MD201 XL 1.5L MT 2024
Skipping CHANGAN MD201 XL 1.5L MT 2024 due to invalid Version URL
Processing model: CHANGAN MS 201 MT 2024
Skipping CHANGAN MS 201 MT 2024 due to invalid Version URL
Processing model: CHANGAN MS 301 PLUS AC ESP 2024
Skipping CHANGAN MS 301 PLUS AC ESP 2024 due to invalid Version URL
Processing model: CHEVROLET COLORADO HIGH COUNTRY 4X4 AT 2024
Processing model: CHEVROLET COLORADO LS 4X2 MT 2024
Processing model: CHEVROLET COLORADO LT 4X4 AT 2024
Processing model: CHEVROLET COLORADO LT 4X4 MT 2024
Processing model: CHEVROLET COLORADO LTZ 4X4 AT 2024
Processi